# Exercicio PySpark

Exercicio proposto para a aula de Storage.

In [1]:
!pip install pyspark findspark

In [59]:
!hdfs dfsadmin -safemode leave
!hdfs dfs -mkdir /user/root/
!hdfs dfs -put /mnt/notebooks/src/titanic-dataset.csv /user/root/titanic-dataset.csv

Safe mode is OFF
mkdir: `/user/root': File exists
put: `/user/root/titanic-dataset.csv': File exists


In [60]:
import findspark
findspark.init()
findspark.find()

'/opt/spark'

In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [62]:
from pprint import pprint

In [84]:
df = spark.read.format("csv").option("header","true").load("titanic-dataset.csv")

### Renomeando colunas para ficar mais amigável

In [87]:
df = df.withColumnRenamed(
    "PassengerId", "passenger_id"
).withColumnRenamed(
    "Survived", "survived"
).withColumnRenamed(
    "Pclass", "passenger_class"
).withColumnRenamed(
    "Sex", "sex"
).withColumnRenamed(
    "Age", "age"
).withColumnRenamed(
    "SibSp", "siblings_and_spouses"
).withColumnRenamed(
    "Parch", "number_of_parents_or_children_onboard"
).withColumnRenamed(
    "Ticket", "ticket"
).withColumnRenamed(
    "Fare", "fare"
).withColumnRenamed(
    "Cabin", "cabin"
).withColumnRenamed(
    "Embarked", "port_of_origin"
).withColumn(
    "port_of_origin", when(
        col("port_of_origin") == "C","Cherbourg"
    ).when(
        col("port_of_origin") == "Q","Queenstown"
    ).when(
        col("port_of_origin") == "S","Southampton"
    )
).withColumn(
    "survived", when(
        col("survived") == 1, True
    ).when(
        col("survived") == 0, False
    )
)

### Analise do dataset

In [88]:
df.limit(5).show() # Preview do dataset

+------------+--------+---------------+--------------------+------+---+--------------------+-------------------------------------+----------------+-------+-----+--------------+
|passenger_id|survived|passenger_class|                Name|   sex|age|siblings_and_spouses|number_of_parents_or_children_onboard|          ticket|   fare|cabin|port_of_origin|
+------------+--------+---------------+--------------------+------+---+--------------------+-------------------------------------+----------------+-------+-----+--------------+
|           1|   false|              3|Braund, Mr. Owen ...|  male| 22|                   1|                                    0|       A/5 21171|   7.25| null|   Southampton|
|           2|    true|              1|Cumings, Mrs. Joh...|female| 38|                   1|                                    0|        PC 17599|71.2833|  C85|     Cherbourg|
|           3|    true|              3|Heikkinen, Miss. ...|female| 26|                   0|                       

In [95]:
df.printSchema() # Schema do modelo

root
 |-- passenger_id: string (nullable = true)
 |-- survived: boolean (nullable = true)
 |-- passenger_class: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- siblings_and_spouses: string (nullable = true)
 |-- number_of_parents_or_children_onboard: string (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- port_of_origin: string (nullable = true)



In [90]:
df.groupBy(col("survived")).count().show() # Quantidade de pessoas que sobreviveram ao Titanic

+--------+-----+
|survived|count|
+--------+-----+
|    true|  342|
|   false|  549|
+--------+-----+



In [91]:
df.groupBy(col("survived"), col("sex")).count().show() # Quantidade de pessoas que sobreviveram ao Titanic

+--------+------+-----+
|survived|   sex|count|
+--------+------+-----+
|   false|female|   81|
|    true|  male|  109|
|    true|female|  233|
|   false|  male|  468|
+--------+------+-----+



In [92]:
df.groupBy(col("survived"), col("sex"), col("passenger_class")).count().show()

+--------+------+---------------+-----+
|survived|   sex|passenger_class|count|
+--------+------+---------------+-----+
|   false|  male|              1|   77|
|    true|  male|              2|   17|
|   false|female|              1|    3|
|   false|female|              2|    6|
|    true|female|              2|   70|
|    true|  male|              3|   47|
|   false|female|              3|   72|
|    true|  male|              1|   45|
|   false|  male|              2|   91|
|    true|female|              3|   72|
|   false|  male|              3|  300|
|    true|female|              1|   91|
+--------+------+---------------+-----+



In [93]:
df.groupBy(col("survived"), col("sex"), col("port_of_origin")).count().show()

+--------+------+--------------+-----+
|survived|   sex|port_of_origin|count|
+--------+------+--------------+-----+
|   false|  male|    Queenstown|   38|
|    true|female|    Queenstown|   27|
|    true|female|   Southampton|  140|
|   false|  male|     Cherbourg|   66|
|    true|  male|   Southampton|   77|
|   false|  male|   Southampton|  364|
|    true|female|     Cherbourg|   64|
|    true|  male|     Cherbourg|   29|
|    true|  male|    Queenstown|    3|
|    true|female|          null|    2|
|   false|female|    Queenstown|    9|
|   false|female|   Southampton|   63|
|   false|female|     Cherbourg|    9|
+--------+------+--------------+-----+

